In [1]:
import numpy as np
from collections import defaultdict
import itertools
from itertools import product

<b> Step 1. Calculate transition and emission probabilities </b>

In [45]:
def build_lists_and_probabilities():

    # Processing training data and create count sets

    set_of_tags = set()
    set_of_tags.add("<s>"); set_of_tags.add("</s>")
    set_of_words = set()

    # Get counts of tag, words, word-tags
    word_tag_count = defaultdict(int) # C(t, w)
    word_count = defaultdict(int) # C(w)
    tag_count = defaultdict(int) # C(t)

    tag_sequence = ["<s>"]  # Sequence of tags, a sentence ends with .
    tag_sequence_count = 0 
    tag_tag_count = defaultdict(int) # C(t_{i-1}, t_i)

    with open('berp-POS-training.txt', 'r') as train_file:
        train_data = train_file.readlines()

    # print('{}'.format(train_data[0][2:]))

    for line in train_data:
         if line != '\n':
            word_tag = line.strip().split('\t')
            word = word_tag[1]
            set_of_tags.add(word_tag[2])
            set_of_words.add(word_tag[1])

            word_tag_count[(word_tag[1], word_tag[2])] += 1
            word_count[word_tag[1]] += 1
            tag_count[word_tag[2]] += 2

            # Get all tags from all lines
            tag_sequence.append(word_tag[2])
            if word_tag[1] == ".":
                tag_sequence.append("</s>")
                for i in range(0, len(tag_sequence)-1):
                    tag_tag_count[(tag_sequence[i], tag_sequence[i+1])] += 1
                tag_sequence = ["<s>"]
                tag_sequence_count += 1

    tag_count["<s>"] = tag_sequence_count
    tag_count["</s>"] = tag_sequence_count


    # Combine tag set and word set and 0 count in word_tag_count if not already there 
    all_word_tag_combos = list(product(set_of_words, set_of_tags))
    for word_tag_combo in all_word_tag_combos:
        if word_tag_combo not in word_tag_count.keys():
            word_tag_count[word_tag_combo] = 0

    # Same thing for tag_tag_count
    all_tag_tag_combos = list(product(set_of_tags, set_of_tags))
    for tag_tag_combo in all_tag_tag_combos:
        if tag_tag_combo not in tag_tag_count.keys():
            tag_tag_count[tag_tag_combo] = 0

    # Get lists and probs
    transition_probs = calculate_transition(tag_tag_count, tag_count)
    # emission_probs = laplace_smooth_emission(word_tag_count, tag_count, word_count) # Smooth or not
    emission_probs = calculate_emission(word_tag_count, tag_count)
    list_of_tags = list(sorted(set_of_tags))
    list_of_words = list(sorted(set_of_words))
    
    return list_of_tags, word_count, transition_probs, emission_probs

In [21]:
def calculate_transition(tag_tag_count, tag_count):
    # Calculate transition probabilities
    # P(t_i | t_{i-1}) = C(t_{i-1, t_i}) / C(t_{i-1})
    transition_probs = {}
    for tag_tag in tag_tag_count:
        count_tag1tag2 = tag_tag_count[tag_tag]
        count_tag1 = tag_count[tag_tag[0]]
        transition_probs[tag_tag] = count_tag1tag2 / count_tag1        
    return transition_probs

In [22]:
def calculate_emission(word_tag_count, tag_count):
    # Calculate emission probabilities
    # P(w | t) = C(t, w) / C(t)
    # word-tag pairings with 0 prob not included, will be 0 by default if not in dict
    # No smoothing yet
    emission_probs = {}
    for word_tag in word_tag_count:
        count_word_tag = word_tag_count[word_tag]
        count_tag = tag_count[word_tag[1]]
        emission_probs[word_tag] = count_word_tag / count_tag
    return emission_probs

In [35]:
def laplace_smooth_emission(word_tag_count, tag_count, word_count):
    vocab_size = len(word_count)
    
    emission_probs = {}
    for word_tag in word_tag_count:
        count_word_tag = word_tag_count[word_tag]
        count_tag = tag_count[word_tag[1]]
        emission_probs[word_tag] = (count_word_tag + 1) / (count_tag + vocab_size)
    return emission_probs

<b>Dealing with unseen data in testset:</b>

Create a placeholder UNK class in both matrices? 


<b> Step 2. Viterbi algorithm </b>

In [46]:
def viterbi_pos_tagger(list_of_tags, input_sentence, transition_probs, emission_probs):
    # Initializing matrices & vector
    # Matrix with tags vs words
    p = np.zeros(shape=(len(list_of_tags), len(input_sentence)))
    # Matrix for backtrace
    back = np.zeros(shape=(len(list_of_tags), len(input_sentence)), dtype=np.int)

    # Initializing step: P(tag|start) * P(word1|tag)
    for tag_i, tag in enumerate(list_of_tags):
        # Fill first col of matrix p & back matrices
        tag_given_start = ('<s>', tag)
        word_given_tag = (input_sentence[0], tag)
        p[tag_i, 0] = transition_probs[tag_given_start] * emission_probs[word_given_tag]
        back[tag_i, 0] = 0  # RECHECK this - not sure how to initialize back pointer

    # Recursion step - go through every tag for each token:
    for word_i in range(1, len(input_sentence)):
        for tagi1, tag1 in enumerate(list_of_tags):
            # For each tag, get its prob given all other tags:
            # Prev column * P(tag|all tags) * P(word|tag)
            # Fill in viterbi matrix
            p[tagi1, word_i] = np.max([p[tagi2, word_i - 1] * transition_probs[tag2, tag1] * emission_probs[input_sentence[word_i], tag1] for tagi2,tag2 in enumerate(list_of_tags)])
            # Fill in backpointer
            back[tagi1, word_i] = np.argmax([p[tagi2, word_i - 1] * transition_probs[tag2, tag1] * emission_probs[input_sentence[word_i], tag1] for tagi2,tag2 in enumerate(list_of_tags)])

    # Termination steps
    best_path_prob = np.max([p[tag_i, len(input_sentence)-1] for tag_i, tag in enumerate(list_of_tags)])        
    best_path_pointer = np.argmax([p[tag_i, len(input_sentence)-1] for tag_i, tag in enumerate(list_of_tags)])        

    return p, back, best_path_pointer, best_path_prob

In [47]:
def backtrace(back, best_path_pointer):
    path_idx = [best_path_pointer]
    for column_i, column in enumerate(back.T[::-1]): # Starts at end 
        max_tag_idx = max(column)
        path_idx.append(max_tag_idx)
    # print(path_idx)

    tag_seq = []
    for i in range(0,len(path_idx)-1):
        tag_seq.append(list_of_tags[path_idx[i]])

    print('Input sentence: {}'.format(input_sentence))
    print('Part of speech: {}'.format(tag_seq[::-1]))
    
    return tag_seq

<b> Testing time </b>

In [48]:
input_sentence = ['how', 'late', 'is', 'cha', '-', 'am', 'open', '.']

list_of_tags, word_count, transition_probs, emission_probs = build_lists_and_probabilities()
p, back, best_path_pointer, best_path_prob = viterbi_pos_tagger(list_of_tags, input_sentence, transition_probs, emission_probs)
output_tags = backtrace(back, best_path_pointer)

Input sentence: ['how', 'late', 'is', 'cha', '-', 'am', 'open', '.']
Part of speech: ['WRB', 'JJ', 'VBZ', 'NN', 'HYPH', 'NN', 'JJ', '.']


Input sentence: ['i', "'d", 'like', 'to', 'go', 'to', 'a', 'fancy', 'restaurant', '.']
Part of speech: ['PRP', 'MD', 'VB', 'TO', 'VB', 'IN', 'DT', 'JJ', 'NN', '.']